In [1]:
import numpy as np
import warnings
import tensorflow as tf
import os
import sys
warnings.simplefilter('ignore')
sys.path.insert(0,'../bert')
import pandas as pd
from io import StringIO
import io
import run_classifier_inmem_noexport
from tensorflow.saved_model import tag_constants 
import pickle 
from bert import tokenization

global NUM_CLASS

NUM_CLASS=150

flags = tf.flags
FLAGS = flags.FLAGS

/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: 

In [2]:
def getSess():
    flags = tf.flags
    FLAGS = flags.FLAGS

    flags.DEFINE_string(
        "export_dir", '../../export_models/percent90/',
        "The dir where the exported model has been written.")
    
    tokenizer = tokenization.FullTokenizer(vocab_file='../../embeddings/uncased_L-12_H-768_A-12/vocab.txt', do_lower_case=True)
    sess = tf.InteractiveSession()
    tf.saved_model.loader.load(sess, [tag_constants.SERVING], FLAGS.export_dir)
    return sess, tokenizer

In [3]:
def predict_inmem(sess, FLAGS, tokenizer, question):
    global NUM_CLASS
    def append_feature(feature):
        eval_features.append(feature)
        eval_writer.process_feature(feature)
        
    examples = run_classifier_inmem_noexport.getinput(question)    
    tmp_file = os.path.join('/tmp/', "eval.tf_record")
    label_list = [str(x) for x in range(NUM_CLASS)]
    
    # write test sentence to tmp_file
    run_classifier_inmem_noexport.file_based_convert_examples_to_features(examples, label_list, 128, tokenizer, tmp_file)    
    
    predict_input_fn = run_classifier_inmem_noexport.file_based_input_fn_builder(
        input_file=tmp_file,
        seq_length=128,
        is_training=False,
        drop_remainder=False)


    record_iterator = tf.python_io.tf_record_iterator(path=tmp_file)    
    
    graph = tf.get_default_graph()
    tensor_input_ids = graph.get_tensor_by_name('input_ids_1:0')
    tensor_input_mask = graph.get_tensor_by_name('input_mask_1:0')
    tensor_label_ids = graph.get_tensor_by_name('label_ids_1:0')
    tensor_segment_ids = graph.get_tensor_by_name('segment_ids_1:0')
    tensor_outputs = graph.get_tensor_by_name('loss/Softmax:0')
    
    for sentence in record_iterator:
        example = tf.train.Example()
        example.ParseFromString(sentence)

        unique_ids = example.features.feature['unique_ids'].int64_list.value
        input_ids = example.features.feature['input_ids'].int64_list.value
        input_mask = example.features.feature['input_mask'].int64_list.value
        segment_ids = example.features.feature['segment_ids'].int64_list.value
        label_ids = example.features.feature['label_ids'].int64_list.value


        result = sess.run([tensor_outputs],feed_dict={
            tensor_input_ids: np.array(input_ids).reshape(-1, 128),
            tensor_input_mask: np.array(input_mask).reshape(-1, 128),
            tensor_segment_ids: np.array(segment_ids).reshape(-1, 128),
            tensor_label_ids: np.array(label_ids).reshape(-1, 1)
        })  

        return result[0][0]

In [4]:
global session
tf.app.flags.DEFINE_string('f', '', 'kernel')
session, tokenizer = getSess()
r = predict_inmem(session, FLAGS, tokenizer, 'How do you like the weather today?')


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../export_models/percent90/variables/variables
INFO:tensorflow:Writing example 0 of 1
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] how do you like the weather today ? [SEP]
INFO:tensorflow:input_ids: 101 2129 2079 2017 2066 1996 4633 2651 1029 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [5]:
r

array([0.00558537, 0.00369703, 0.00468001, 0.00498081, 0.0034646 ,
       0.00451047, 0.00282442, 0.0040466 , 0.00530498, 0.00895857,
       0.00584554, 0.00578684, 0.00510342, 0.00506804, 0.00249405,
       0.00756115, 0.00357978, 0.00403121, 0.00266262, 0.00395891,
       0.00523536, 0.00659561, 0.00527991, 0.01044015, 0.00446732,
       0.00421282, 0.00630595, 0.00785989, 0.00552344, 0.00879134,
       0.00563308, 0.00657231, 0.00400539, 0.00746097, 0.00917526,
       0.00309761, 0.00461051, 0.00460911, 0.01036947, 0.00547472,
       0.00421908, 0.0060829 , 0.00371816, 0.00331775, 0.00961806,
       0.00519314, 0.00485394, 0.00579598, 0.00723409, 0.00923276,
       0.00623235, 0.00673085, 0.01282593, 0.00733177, 0.01134854,
       0.00794534, 0.00593088, 0.0044748 , 0.00737451, 0.00362329,
       0.00319522, 0.0088563 , 0.00614929, 0.00430552, 0.00204635,
       0.00412285, 0.00960202, 0.00883069, 0.0056582 , 0.00431228,
       0.00727059, 0.00426049, 0.00588124, 0.00618076, 0.00730